### Cell 1: Imports and Loading data

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# Define Paths
DATA_PATH = "../data/raw/ethiopia_fi_unified_data.xlsx"
CODES_PATH = "../data/raw/reference_codes.xlsx"
GUIDE_PATH = "../data/raw/Additional Data Points Guide.xlsx"

# Load Data
# Task 1 specifies Sheet 1 is 'data' and Sheet 2 is 'impact_links'
try:
    df_data = pd.read_excel(DATA_PATH, sheet_name='ethiopia_fi_unified_data')
    df_impact = pd.read_excel(DATA_PATH, sheet_name='Impact_sheet')
    codes = pd.read_excel(CODES_PATH)
    
    # We combine them for exploration if they share the same columns 
    # as per instructions: "all records share the same columns"
    df = pd.concat([df_data, df_impact], ignore_index=True)
    
    print("--- Data Loaded Successfully from Excel ---")
    print(f"Unified Dataset Shape: {df.shape}")
except Exception as e:
    print(f"Error loading Excel files: {e}")
    print("Ensure you have installed 'openpyxl' via pip or conda.")

--- Data Loaded Successfully from Excel ---
Unified Dataset Shape: (57, 35)


C:\Users\user\AppData\Local\Temp\ipykernel_25960\930431876.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df_data, df_impact], ignore_index=True)


### Cell 2: Exploration - Summary Counts

In [2]:
print("--- Counts by Record Type ---")
print(df['record_type'].value_counts())

print("\n--- Counts by Pillar ---")
# Using dropna=False to see events/impact links that might have empty pillars
print(df['pillar'].value_counts(dropna=False))

print("\n--- Confidence Levels ---")
print(df['confidence'].value_counts())

--- Counts by Record Type ---
record_type
observation    30
impact_link    14
event          10
target          3
Name: count, dtype: int64

--- Counts by Pillar ---
pillar
ACCESS           20
USAGE            17
NaN              10
GENDER            6
AFFORDABILITY     4
Name: count, dtype: int64

--- Confidence Levels ---
confidence
high      44
medium    13
Name: count, dtype: int64


### Cell 3: Exploration - Temporal Range & Indicators

In [3]:
df['observation_date'] = pd.to_datetime(df['observation_date'], errors='coerce')
print(f"Observation Temporal Range: {df['observation_date'].min()} to {df['observation_date'].max()}")

print("\n--- Unique Indicator Codes ---")
print(df[df['record_type']=='observation']['indicator_code'].unique())

Observation Temporal Range: 2014-12-31 00:00:00 to 2030-12-31 00:00:00

--- Unique Indicator Codes ---
['ACC_OWNERSHIP' 'ACC_MM_ACCOUNT' 'ACC_4G_COV' 'ACC_MOBILE_PEN'
 'ACC_FAYDA' 'USG_P2P_COUNT' 'USG_P2P_VALUE' 'USG_ATM_COUNT'
 'USG_ATM_VALUE' 'USG_CROSSOVER' 'USG_TELEBIRR_USERS' 'USG_TELEBIRR_VALUE'
 'USG_MPESA_USERS' 'USG_MPESA_ACTIVE' 'USG_ACTIVE_RATE' 'AFF_DATA_INCOME'
 'GEN_GAP_ACC' 'GEN_MM_SHARE' 'GEN_GAP_MOBILE']


### Cell 4: Data Enrichment Function and Define New Records

In [4]:
# Data Enrichment Function
def enrich_dataset(original_df, new_records_list):
    """Appends new records to the dataframe following the schema."""
    new_data = pd.DataFrame(new_records_list)
    enriched_df = pd.concat([original_df, new_data], ignore_index=True)
    return enriched_df

# Fully Detailed Enrichment to support Task 1 Documentation & Task 3 Modeling
new_entries = [
    # 1. ADDITIONAL OBSERVATIONS (Smartphone & Findex Proxy)
    {
        'record_type': 'observation',
        'pillar': 'INFRASTRUCTURE', 
        'indicator': 'Smartphone Ownership',
        'indicator_code': 'INF_SMP_OWN',
        'value_numeric': 43.0,
        'observation_date': '2023-09-30',
        'source_name': 'BII Report',
        'source_url': 'https://assets.bii.co.uk/wp-content/uploads/2024/10/09105903/BII-Impact-of-investment-in-the-Ethiopian-telecoms-market_2024.pdf',
        'confidence': 'high',
        'collected_by': 'Rufta',
        'collection_date': datetime.now().strftime('%Y-%m-%d'),
        'notes': 'Added to test the hypothesis that hardware costs are a bottleneck for mobile app adoption.'
    },
    
    # 2. ADDITIONAL EVENTS (The "Shocks")
    # Note: Per instructions, event records should have a 'category' but the 'pillar' field is typically left empty 
    # to avoid bias, as they can affect multiple pillars.
    {
        'record_type': 'event',
        'category': 'product_launch',
        'pillar': np.nan, 
        'indicator': 'Telebirr Launch',
        'indicator_code': 'EVT_TELEBIRR',
        'observation_date': '2021-05-11',
        'source_name': 'Ethio Telecom',
        'source_url': 'https://www.ethiotelecom.et/telebirr/',
        'confidence': 'high',
        'collected_by': 'Rufta',
        'collection_date': datetime.now().strftime('%Y-%m-%d'),
        'notes': 'Correlated with an immediate spike in mobile money penetration[cite: 499].'
    },
    {
        'record_type': 'event',
        'category': 'market_entry',
        'pillar': np.nan,
        'indicator': 'Safaricom M-Pesa Launch',
        'indicator_code': 'EVT_MPESA',
        'observation_date': '2023-08-15',
        'source_name': 'Safaricom Ethiopia',
        'source_url': 'https://www.safaricom.et/personal/m-pesa',
        'confidence': 'high',
        'collected_by': 'Rufta',
        'collection_date': datetime.now().strftime('%Y-%m-%d'),
        'notes': 'Acted as a leading indicator for transaction volume growth seen in 2023[cite: 500].'
    },
    {
        'record_type': 'event',
        'category': 'policy',
        'pillar': np.nan,
        'indicator': 'National ID (Fayda) Rollout',
        'indicator_code': 'EVT_FAYDA',
        'observation_date': '2024-06-01',
        'source_name': 'NID Ethiopia',
        'source_url': 'https://id.et/',
        'confidence': 'medium',
        'collected_by': 'Rufta',
        'collection_date': datetime.now().strftime('%Y-%m-%d'),
        'notes': 'Logged as a catalyst event to model future reductions in bank KYC friction[cite: 532].'
    },

    # 3. ADDITIONAL IMPACT LINKS (Crucial for the Matrix)
    # Impact links MUST have a pillar to define which area of inclusion they affect.
    {
        'record_type': 'impact_link',
        'parent_id': 'EVT_TELEBIRR',
        'pillar': 'USAGE', 
        'related_indicator': 'ACC_MM_ACCOUNT',
        'impact_direction': 'positive',
        'impact_magnitude': 'high',
        'lag_months': 0,
        'evidence_basis': 'Telebirr reached 4M users in weeks.',
        'collected_by': 'Rufta',
        'collection_date': datetime.now().strftime('%Y-%m-%d')
    },
    {
        'record_type': 'impact_link',
        'parent_id': 'EVT_TELEBIRR',
        'pillar': 'USAGE',
        'related_indicator': 'USG_P2P_VALUE',
        'impact_direction': 'positive',
        'impact_magnitude': 'high',
        'lag_months': 0,
        'evidence_basis': 'First interoperable P2P system[cite: 566].',
        'collected_by': 'Rufta',
        'collection_date': datetime.now().strftime('%Y-%m-%d')
    },
    {
        'record_type': 'impact_link',
        'parent_id': 'EVT_MPESA',
        'pillar': 'USAGE',
        'related_indicator': 'ACC_MM_ACCOUNT',
        'impact_direction': 'positive',
        'impact_magnitude': 'medium',
        'lag_months': 0,
        'evidence_basis': 'M-Pesa entry increased competition and adoption[cite: 498].',
        'collected_by': 'Rufta',
        'collection_date': datetime.now().strftime('%Y-%m-%d')
    },
    {
        'record_type': 'impact_link',
        'parent_id': 'EVT_FAYDA',
        'pillar': 'ACCESS',
        'related_indicator': 'ACC_OWNERSHIP',
        'impact_direction': 'positive',
        'impact_magnitude': 'high',
        'lag_months': 12,
        'evidence_basis': 'NID is a requirement for KYC-compliant accounts; reduces friction[cite: 482].',
        'collected_by': 'Rufta',
        'collection_date': datetime.now().strftime('%Y-%m-%d')
    }
]

### Cell 5: Save Enriched Data

In [5]:
df_final = enrich_dataset(df, new_entries)
output_path = "../data/processed/ethiopia_fi_enriched.csv"
os.makedirs("../data/processed", exist_ok=True)
df_final.to_csv(output_path, index=False)
print(f"Task 1 Complete: Enriched data saved to {output_path}")

Task 1 Complete: Enriched data saved to ../data/processed/ethiopia_fi_enriched.csv
